In [1]:
%load_ext autoreload
%autoreload 2
import os
from tqdm import tqdm

from modules.constants import *
from modules.utilities import *
from modules.text_classifiers import *

random_sate = 5

In [2]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [23]:
def filter_performance_topics(filename, columns,preprocessor, classifier):
    df = read_aidev(filename)
    output_rows = []

    for _, row in tqdm(df.iterrows()):
        text = ""
        for col in columns:
            if isinstance(row[col], str):
                if text != "":
                    text += "\n"
                
                val = row[col]
                if preprocessor is not None:
                    val = preprocessor.preprocess(val)

                text += val

        if text and text != "" and classifier.classify(text):
            output_rows.append(row)
            try:
                row["matched_words"] = classifier.get_matches()
            except:
                pass

    output_df = pd.DataFrame(output_rows)
    
    return output_df

In [ ]:
filename = FileName.POP_ISSUE
columns = ["title", "body"]
keywords = set(["performance", "compile-time-hog", "perf", "hang", "optimization", "responsive", "slow" , "speed", "latency", "throughput", "wait", "slow", "fast", "lag", "tim", "minor", "stuck", "instant", "respons", "react", "speed", "latenc", "perform", "throughput", "hang", "memory", "leak"])

preprocessor = None
classifier = RegexClassifier(keywords=keywords)

filter_performance_topics(filename, columns, preprocessor, classifier)


4614it [00:01, 3703.72it/s]


,id,number,title,body,user,state,created_at,closed_at,html_url,matched_words
1,3164229012,94,Workflows Coming Soon - tools reimagined,### Project Version\n\n5.5.0\n\n### Bug Descri...,guidedways,closed,2025-06-20T19:42:16Z,2025-06-20T20:08:12Z,https://github.com/BeehiveInnovations/zen-mcp-...,[performs]
3,3208181218,129,架构优化：统一项目路径管理体系 - 合并DirectoryService与ProjectMa...,# 🎯 架构优化：统一项目路径管理体系\n\n## 🔍 问题描述\n\n当前PromptX在...,deepracticexs,closed,2025-07-07T09:12:24Z,2025-07-07T11:05:45Z,https://github.com/Deepractice/PromptX/issues/129,[optimization]
6,3200531167,4301,`Pkg.instantiate` shouldn't need to uncompress...,(This is not specific to `Pkg.instantiate` rea...,KristofferC,closed,2025-07-03T20:15:07Z,2025-07-04T08:34:05Z,https://github.com/JuliaLang/Pkg.jl/issues/4301,"[instantiate, fast, instantiate]"
8,3244979480,1018,Implement context engineering,Implement below in src/praisonai-agents/praiso...,MervinPraison,closed,2025-07-19T06:14:41Z,2025-07-19T08:45:52Z,https://github.com/MervinPraison/PraisonAI/iss...,"[Performance, perform, responsibility, FastAPI..."
10,3160777773,986,feat: Implement local-first chat persistence w...,## Summary\n\nImplement local-first chat persi...,AtlantisPleb,closed,2025-06-19T15:52:01Z,2025-06-20T01:21:19Z,https://github.com/OpenAgentsInc/openagents/is...,"[Performance, reactive, time, timestamps, time..."
...,...,...,...,...,...,...,...,...,...,...
4602,2372703051,8599,Package scoping fails when workspace glob has ...,### Verify canary release\r\n\r\n- [X] I verif...,timostamm,open,2024-06-25T13:26:28Z,None,https://github.com/vercel/turborepo/issues/8599,"[timostamm, timostamm]"
4603,3028076678,10403,`crates/turborepo-scm/src/git.rs` panics in Gi...,### Verify canary release\n\n- [ ] I verified ...,ScarletFlash,open,2025-04-29T12:07:43Z,None,https://github.com/vercel/turborepo/issues/10403,"[memory, slower]"
4605,2911626727,1906,Replace react-scripts with Vite,None,whitphx,closed,2025-03-11T19:16:18Z,2025-03-12T09:02:41Z,https://github.com/whitphx/streamlit-webrtc/is...,[react]
4607,2327353475,3110,Picker prop 'migrate' warning despite not usin...,<!--\r\nNOTE: please submit only bug reports h...,king960,closed,2024-05-31T09:21:35Z,2025-02-25T00:34:53Z,https://github.com/wix/react-native-ui-lib/iss...,"[react, react, React, React, React]"


In [ ]:
filename = FileName.POP_PULL_REQUEST
columns = ["title", "body"]
keywords = set(["performance", "compile-time-hog", "perf", "hang", "optimization", "responsive", "slow" , "speed", "latency", "throughput", "wait", "slow", "fast", "lag", "tim", "minor", "stuck", "instant", "respons", "react", "speed", "latenc", "perform", "throughput", "hang", "memory", "leak"])

preprocessor = None
classifier = RegexClassifier(keywords=keywords)

filter_performance_topics(filename, columns, preprocessor, classifier)


33596it [00:07, 4398.32it/s]


,id,number,title,body,agent,user_id,user,state,created_at,closed_at,merged_at,repo_id,repo_url,html_url,matched_words
0,3264933329,2911,Fix: Wait for all partitions in load_collectio...,## Summary\n\nFixes an issue where `load_colle...,Claude_Code,108661493,weiliu1031,closed,2025-07-26T02:59:01Z,2025-07-29T07:01:20Z,None,191751505,https://api.github.com/repos/milvus-io/pymilvus,https://github.com/milvus-io/pymilvus/pull/2911,"[Wait, waiting, wait_for_loading_collection, t..."
2,3265640341,30,Add build staleness detection for debug CLI,## Summary\r\n\r\n Implements comprehensive b...,Claude_Code,7475,MSch,closed,2025-07-26T13:31:19Z,2025-07-26T13:37:22Z,2025-07-26T13:37:22Z,988488798,https://api.github.com/repos/steipete/Peekaboo,https://github.com/steipete/Peekaboo/pull/30,"[time, timestamp, time, timestamp, timestamp, ..."
3,3265709660,205,feat: add comprehensive README screenshots wit...,## Type of Change\n\n- [ ] 🐛 `bug` - Bug fix (...,Claude_Code,80381,sugyan,closed,2025-07-26T14:07:22Z,2025-07-26T14:45:30Z,2025-07-26T14:45:30Z,999285986,https://api.github.com/repos/sugyan/claude-cod...,https://github.com/sugyan/claude-code-webui/pu...,"[performance, Performance, Responsive, Respons..."
13,3231955292,32664,feat: finalize Swagger annotations implementat...,## Summary\nFinal cleanup and finalization of ...,Claude_Code,1236198,spbolton,open,2025-07-15T11:48:51Z,None,None,3729629,https://api.github.com/repos/dotCMS/core,https://github.com/dotCMS/core/pull/32664,[performing]
15,3232844270,359,feat: Implement asymmetric VMEC support (lasym...,## Summary\nImplements full asymmetric VMEC su...,Claude_Code,149655,krystophny,open,2025-07-15T16:02:35Z,None,None,887571250,https://api.github.com/repos/proximafusion/vmecpp,https://github.com/proximafusion/vmecpp/pull/359,"[Memory, performance]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33583,3260345989,622,Match dividend check pattern to invoice requir...,# Match dividend check pattern to invoice requ...,Devin,158243242,devin-ai-integration[bot],closed,2025-07-24T15:37:09Z,2025-07-25T21:22:00Z,2025-07-25T21:22:00Z,955904085,https://api.github.com/repos/antiwork/flexile,https://github.com/antiwork/flexile/pull/622,"[Minor, minor, minor]"
33584,3260452571,1542,Fix CSS color assertions in test_lambdas.py,# Fix CSS color assertions in test_lambdas.py\...,Devin,158243242,devin-ai-integration[bot],closed,2025-07-24T16:14:38Z,2025-07-25T11:29:31Z,2025-07-25T11:29:31Z,577975610,https://api.github.com/repos/reflex-dev/reflex...,https://github.com/reflex-dev/reflex-web/pull/...,"[Minor, minor, minor]"
33588,2857171695,1411,refactor: Port measurement overlay to React,Restores the alt to measure behavior using Rea...,Devin,158243242,devin-ai-integration[bot],closed,2025-02-17T08:46:25Z,2025-02-18T04:51:00Z,2025-02-18T04:51:00Z,820087727,https://api.github.com/repos/onlook-dev/onlook,https://github.com/onlook-dev/onlook/pull/1411,"[React, React, React, React]"
33589,2857279950,8459,Add missing OpenSSL TLSEXT status response codes,Fixes the build failure in the OpenSSL coexist...,Devin,158243242,devin-ai-integration[bot],closed,2025-02-17T09:32:13Z,2025-02-17T09:34:11Z,None,28928522,https://api.github.com/repos/wolfSSL/wolfssl,https://github.com/wolfSSL/wolfssl/pull/8459,"[response, response, response]"


In [24]:
filename = FileName.POP_PULL_REQUEST
columns = ["title", "body"]
keywords = set(["performance", "compile-time-hog", "perf", "hang", "optimization", "responsive", "slow" , "speed", "latency", "throughput", "wait", "slow", "fast", "lag", "tim", "minor", "stuck", "instant", "respons", "react", "speed", "latenc", "perform", "throughput", "hang", "memory", "leak"])

preprocessor = None
base_text = ", ".join(keywords)
classifier = EmbeddingClassifier(base_text, similarity_threshold=0.4, embedding_model="google/embeddinggemma-300m")

filter_performance_topics(filename, columns, preprocessor, classifier)


33596it [25:59, 21.54it/s]


,id,number,title,body,agent,user_id,user,state,created_at,closed_at,merged_at,repo_id,repo_url,html_url
55,3220367089,936,Fix race conditions in myaudio package,## Summary\n- Fixed all 16 race conditions in ...,Claude_Code,7030001,tphakala,closed,2025-07-10T19:01:53Z,2025-07-10T19:07:35Z,2025-07-10T19:07:35Z,707764474,https://api.github.com/repos/tphakala/birdnet-go,https://github.com/tphakala/birdnet-go/pull/936
102,3200679276,4304,Implement lazy loading for RegistryInstance to...,👨 \r\n\r\nBefore:\r\n\r\n```\r\njulia> @time P...,Claude_Code,1282691,KristofferC,closed,2025-07-03T21:18:03Z,2025-07-04T08:34:04Z,2025-07-04T08:34:04Z,82341193,https://api.github.com/repos/JuliaLang/Pkg.jl,https://github.com/JuliaLang/Pkg.jl/pull/4304
350,3226043406,4902,feat: lazy load CLI command actions for improv...,## Summary\n\nThis PR implements lazy loading ...,Claude_Code,7235481,mldangelo,open,2025-07-13T02:47:49Z,None,None,633927609,https://api.github.com/repos/promptfoo/promptfoo,https://github.com/promptfoo/promptfoo/pull/4902
396,3193198936,841,feat(telemetry): implement performance testing...,## Summary\n\nThis PR implements Phase 8 of th...,Claude_Code,7030001,tphakala,closed,2025-07-01T17:00:54Z,2025-07-01T18:11:08Z,2025-07-01T18:11:08Z,707764474,https://api.github.com/repos/tphakala/birdnet-go,https://github.com/tphakala/birdnet-go/pull/841
513,3109359968,1809,Fix attr cache memory leak by implementing exp...,## Problem\n\nThe attr_cache component was cac...,Copilot,198982749,Copilot,closed,2025-06-02T09:30:10Z,2025-06-30T10:24:57Z,2025-06-30T10:24:57Z,103307180,https://api.github.com/repos/Azure/azure-stora...,https://github.com/Azure/azure-storage-fuse/pu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32633,3114992077,244,feat(editor): implement code splitting and tre...,# Editor Code Splitting and Tree Shaking Optim...,Devin,158243242,devin-ai-integration[bot],closed,2025-06-03T18:22:54Z,2025-06-12T15:23:07Z,None,440017777,https://api.github.com/repos/softmaple/softmaple,https://github.com/softmaple/softmaple/pull/244
32770,3070322024,21374,perf: add p-limit to _getUsersAvailability to ...,# Add p-limit to _getUsersAvailability\n\n## D...,Devin,158243242,devin-ai-integration[bot],closed,2025-05-17T04:28:35Z,2025-05-17T21:05:45Z,2025-05-17T21:05:45Z,350360184,https://api.github.com/repos/calcom/cal.com,https://github.com/calcom/cal.com/pull/21374
32930,2855302194,711,Optimize Font Loading Performance in Tests,# Optimize Font Loading Performance in Tests\n...,Devin,158243242,devin-ai-integration[bot],closed,2025-02-15T07:23:53Z,2025-02-15T07:40:30Z,None,398753576,https://api.github.com/repos/pdfme/pdfme,https://github.com/pdfme/pdfme/pull/711
33328,3240241128,582,Add performance benchmarks requirement to cont...,# Add performance benchmarks requirement to co...,Devin,158243242,devin-ai-integration[bot],closed,2025-07-17T16:47:16Z,2025-07-17T16:50:12Z,2025-07-17T16:50:12Z,955904085,https://api.github.com/repos/antiwork/flexile,https://github.com/antiwork/flexile/pull/582


In [20]:
keywords = set(["performance", "compile-time-hog", "perf", "hang", "optimization", "responsive", "slow" , "speed", "latency", "throughput", "wait", "slow", "fast", "lag", "tim", "minor", "stuck", "instant", "respons", "react", "speed", "latenc", "perform", "throughput", "hang", "memory", "leak"])
base_text = "A compile-time-hog can undermine a system meant to be fast, causing it to hang, lag, or lose its instant responsiveness as latency increases. Even a small leak in memory can turn a minor issue into a serious optimization problem that degrades overall performance. When software fails to react smoothly or remain responsive, it may become slow, drop in speed, get stuck, reduce throughput, and ultimately force users to wait."#", ".join(keywords)
base_text = ", ".join(keywords)
classifier = EmbeddingClassifier(base_text, similarity_threshold=0.4, embedding_model="google/embeddinggemma-300m")

In [21]:
classifier.classify(["Fix: Wait for all partitions in load_collection when some are still loading", "Improve list and collection materializers performance"])

[True, True]

In [19]:
classifier.get_score()

[0.4022243320941925, 0.4155503809452057]

In [ ]:
keywords = set(["performance", "compile-time-hog", "perf", "hang", "optimization", "responsive", "slow" , "speed", "latency", "throughput", "wait", "slow", "fast", "lag", "tim", "minor", "stuck", "instant", "respons", "react", "speed", "latenc", "perform", "throughput", "hang", "memory", "leak"])
base_text = ", ".join(keywords)
prompt = [
    {
        "role": "system",
        "content": "You are text classification agent. Analyze the given text and determine if it talks about software performane. Answer 'performance' or 'non-performance' accordingly.  Performance related text may contain keywords such as: " + ", ".join(keywords)
    }
]
classifier = LLMClassifier(prompt)

In [23]:
classifier.classify(["task: sentence similarity | query: Fix: Wait for all partitions in load_collection when some are still loading", "task: sentence similarity | query: Improve list and collection materializers performance"])

[False, False]

In [ ]:
s